In [1]:
## building an auto-encoder class
## builiding the encoder

6

In [ ]:
from keras import Model 
from keras.layers import Input, Conv2D, ReLU, BatchNormalization, Flatten, Dense, Reshape, Conv2DTranspose, activation
from keras import backend as K
import numpy as np
class Autoencoder:
    
    def __init__(self,
                 input_shape,
                 conv_filters,
                 conv_kernels,
                 conv_strides,
                 latent_space_dim):
        self.input_shape = input_shape # [28,28,1] this means the input is a 28x28 image with 1 channel (grayscale) like mnist
        self.conv_filters = conv_filters #[2,4,8] this means the first layer has two filters, the second 4, and the last 8.
        self.conv_kernels = conv_kernels #[3,5,3] this means the first layer has a 3x3 kernel, the second 5x5, and the last 3x3.
        self.conv_strides= conv_strides  #[1,2,2] this means the first layer has a stride of 1, the second 2, and the last 2.
        self.latent_space_dim = latent_space_dim # 2 this means the bottleneck will only have 2 dimensions.
        self.encoder= None
        self.decoder = None
        self.model= None
        self._num_conv_layers= len(conv_filters)
        self._shape_before_bottleneck=None
        self._build()
        
    def summary(self):
        self.encoder.summary()
        
    def _build(self):
        self._build_encoder()
        self._build_decoder()
        #self._build_autoencoder()
    
    def _build_decoder(self):
        decoder_input= self._add_decoder_input()
        dense_layer= self._add_dense_layer(decoder_input)
        reshape_layer = self._add_reshape_layer(dense_layer)
        conv_transpose_layers= self._add_conv_transpose_layers(reshape_layer)
        decoder_output= self._add_decoder_output(conv_transpose_layers)
        self.decoder=Model(decoder_input, decoder_output, name='decoder')
        
    def _add_decoder_input(self):
        return Input(shape=(self.latent_space_dim,), name='decoder_input')
    
    def _add_dense_layer(self, decoder_input):
        num_neurons = np.prod(self._shape_before_bottleneck)
        ## This will multuply all the elements in the list, because the shape before bottleneck has like 3 elements
        dense_layer = Dense(num_neurons, name='decoder_dense')(decoder_input)
        return dense_layer
    
    def _add_reshape_layer(self, dense_layer):
        ## we want to go back to the three d layer, we want to go the the encoder shape before flattening
        return Reshape(self._shape_before_bottleneck)(dense_layer)
    
    def _add_conv_transpose_layers(self, x):
        ## Add convolutional transpose blocks
        ## Loop through all the conv layers in reverse order and stop at the first layer
        ## we need to ignore the first convolutional layer, and we want to do it in reverse order
        for layer_index in reversed(range(1, self._num_conv_layers)):
            #[0,1,2] ->[2,1]
            x= self._add_conv_transpose_layer(layer_index, x)
        return x
    
    def _add_conv_transpose_layer(self, layer_index, x):
        layer_num = self._num_conv_layers - layer_index
        conv_transpose_layer= Conv2DTranspose(filters=self.conv_filters[layer_index],
                                              kernel_size=self.conv_kernels[layer_index],
                                                strides=self.conv_strides[layer_index],
                                                padding='same',
                                                name=f'decoder_conv_transpose_{layer_num}')
        x= conv_transpose_layer(x)
        x=ReLU(name=f'decoder_relu_{layer_num}')(x)
        x= BatchNormalization(name=f'decoder_bn_{layer_num}')(x)
        return x
         
    def _add_decoder_output(self, x):
        conv_transpose_layer = Conv2DTranspose(filters=1, ## [24 24 1] height and width and num of channels we set to one because of number of channels in mnist
                                              kernel_size=self.conv_kernels[0], ## getting the first kernel size
                                                strides=self.conv_strides[0],
                                                padding='same',
                                                name=f'decoder_conv_transpose_layer_{self._num_conv_layers}')
        x= conv_transpose_layer(x)
        output_layer = activation('sigmoid', name='sigmoid_layer')(x)
         
         
    
    
    def _build_encoder(self):
        encoder_input = self._add_encoder_input()
        conv_layers= self._add_conv_layers(encoder_input)
        bottleneck = self._add_bottleneck(conv_layers)
        self.encoder = Model(encoder_input, bottleneck, name='encoder')
    
    def _add_encoder_input(self):
        return Input(shape=self.input_shape, name='encoder_input')
        
    ## This will create all convolutional blocks in encoder    
    def _add_conv_layers(self, encoder_input):
        x= encoder_input
        for layer_index in range(self._num_conv_layers):
            x = self._add_conv_layer(layer_index, x)
        return x   
    
    def _add_conv_layer(self, layer_index, x):
        layer_number= layer_index+1
        ## Adds a convolutional block to a graph of layers, consisting of a conv 2d+ batch norm + relu 
        conv_layer = Conv2D(filters=self.conv_filters[layer_index],kernel_size=self.conv_kernels[layer_index],
                            strides=self.conv_strides[layer_index], 
                            padding='same', name=f'encoder_conv_layer_{layer_number}')
        x= conv_layer(x)
        x= ReLU(name=f'encoder_relu_{layer_number}')(x)
        x= BatchNormalization(name=f'encoder_bn_{layer_number}')(x)
        return x
    
 
    
    def _add_bottleneck(self, x):
        ## Flatten the data and add bottleneck (a dense layer)
        ## we want to store the shaoe of the data before flattening it, so we can use it in the decoder
        self._shape_before_bottleneck= K.int_shape(x)[1:] #in our case [2 7 7 32] first -> batch size, second and third -> spatial dimensions, fourth -> number of channels
        x= Flatten()(x)
        ## Here it is like we instantiate a dense layer (the first paranthesis) and then we call it on x (with the second paranthesis)
        x= Dense(self.latent_space_dim, name='encoder_output')(x)
        return x
    
if __name__=="__main__":
    autoencoder= Autoencoder(input_shape=[28,28,1], conv_filters=[32,64,64,64], conv_kernels=[3,3,3,3], conv_strides=[1,2,2,1], latent_space_dim=2)
    autoencoder.summary()
    

In [ ]:
## moving to the decoder
 